<a href="https://colab.research.google.com/github/kapibara3chi/ReinforcementLearning/blob/main/ReinfoecementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# simple Sample

## pip

In [1]:
!pip install gym-anytrading

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.9 MB/s eta 0:00:00


## code

In [ ]:
import gym
import gym_anytrading

# 他にもデータ処理や可視化のために必要なライブラリをインポートします
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# CSVファイルから株価データを読み込みます
df = pd.read_csv('stock_data.csv')

# `gym-anytrading`に必要なカラム名に変更します
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Forex環境を構築しますが、株価予測にも同様に適用できます
env = gym.make('forex-v0', df=df, frame_bound=(10, len(df)), window_size=5)


state = env.reset()
while True:
    action = env.action_space.sample()  # ランダムな行動を選択
    next_state, reward, done, info = env.step(action)
    if done:
        print("Info:", info)
        break

plt.figure(figsize=(15, 6))
plt.cla()
env.render_all()
plt.show()